In [3]:
# The code is generated from o3-mini-high

In [4]:
%pip install yfinance

  Using cached yfinance-0.2.52-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached numpy-2.2.2-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached lxml-5.3.0-cp313-cp313-macosx_10_13_universal2.whl.metadata (3.8 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached frozendict-2.4.6-py313-none-any.whl.metadata (23 kB)
  Using cached peewee-3.17.8-cp313-cp313-macosx_15_0_arm64.whl
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached yfinance-0.2.52-py2.py3-none-any.whl (108 kB)
Using cached frozendict-2.4.6-py313-none-any.whl (16 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Using cached lxml-5.3.0-cp313-cp313-macosx_10_13_universal2.whl (8.2 MB)
Using cached multitasking-0.0.11-py3-none-an

In [6]:
# %% [markdown]
"""
# Discounted Cash Flow (DCF) Analysis for NVIDIA (NVDA)

This notebook performs a Discounted Cash Flow (DCF) analysis for NVIDIA using financial data from Yahoo Finance via the `yfinance` package.
If any required variable is not available from the API, please set it manually in the configuration section below.
"""

# %% [markdown]
"""
## 1. Configuration Variables

Set your assumptions and any manual inputs for data that may not be available via the API.
"""

# %%
# Configuration Variables

# Ticker symbol for the company
TICKER = "NVDA"

# Number of years to project free cash flows
projection_years = 5

# Assumed annual free cash flow growth rate (e.g., 10% growth)
cash_flow_growth_rate = 0.10

# Discount rate (WACC) used to discount future cash flows (e.g., 10%)
discount_rate = 0.10

# Terminal growth rate for perpetual growth beyond the projection period (e.g., 3%)
terminal_growth_rate = 0.03

# ---
# If the Free Cash Flow (FCF) data is not available via the API, set a manual value here.
# For example, if you want to use a trailing-twelve-month FCF (in millions or absolute units), set:
manual_free_cash_flow = None  # e.g., 5000

# ---
# Similarly, if total debt, total cash, or shares outstanding are not available, you can manually set them:
manual_total_debt = None        # e.g., 10000
manual_total_cash = None        # e.g., 5000
manual_shares_outstanding = None  # e.g., 2500000000

# %% [markdown]
"""
## 2. Import Libraries and Fetch Company Data

We import the necessary libraries and use `yfinance` to pull the latest financial data for NVIDIA.
"""

# %%
import yfinance as yf
import pandas as pd
import numpy as np

# Fetch data for the given ticker
ticker = yf.Ticker(TICKER)
info = ticker.info

# Display some basic company information
print("Company Name:", info.get("longName", "N/A"))
print("Sector:", info.get("sector", "N/A"))
print("Industry:", info.get("industry", "N/A"))

# %% [markdown]
"""
## 3. Retrieve Free Cash Flow (FCF) Data

We try to obtain the most recent annual Free Cash Flow from the cash flow statement.
If it is missing, the notebook will either use the manually provided value or raise an error.
"""

# %%
# Retrieve the cash flow statement and sort columns so that the most recent period is first.
cash_flow_df = ticker.cashflow
cash_flow_df = cash_flow_df.sort_index(axis=1, ascending=False)

# Check for a row labeled "Free Cash Flow" (or a similar key)
if "Free Cash Flow" in cash_flow_df.index:
    latest_fcf = cash_flow_df.loc["Free Cash Flow"].iloc[0]
elif "FreeCashFlow" in cash_flow_df.index:
    latest_fcf = cash_flow_df.loc["FreeCashFlow"].iloc[0]
else:
    latest_fcf = None

# Use the manual value if API data is not available and a manual value has been set
if latest_fcf is None:
    if manual_free_cash_flow is not None:
        latest_fcf = manual_free_cash_flow
        print("Using manual Free Cash Flow value:", latest_fcf)
    else:
        raise ValueError("Free Cash Flow data is not available via API. Please set 'manual_free_cash_flow' in the configuration.")

print("Latest Free Cash Flow (most recent annual):", latest_fcf)

# %% [markdown]
"""
## 4. Project Future Free Cash Flows

We project the FCF for the next several years using the configured growth rate.
Each projected FCF is then discounted back to its present value.
"""

# %%
projected_fcfs = []
for year in range(1, projection_years + 1):
    projected_fcf = latest_fcf * ((1 + cash_flow_growth_rate) ** year)
    discounted_fcf = projected_fcf / ((1 + discount_rate) ** year)
    projected_fcfs.append({
        "Year": year,
        "Projected FCF": projected_fcf,
        "Discounted FCF": discounted_fcf
    })

fcf_df = pd.DataFrame(projected_fcfs)
print(fcf_df)

# %% [markdown]
"""
## 5. Calculate Terminal Value

At the end of the projection period, we calculate the Terminal Value using the perpetuity growth model.
"""

# %%
# Free Cash Flow at the end of the projection period
fcf_at_end = latest_fcf * ((1 + cash_flow_growth_rate) ** projection_years)

# Terminal Value calculation (undiscounted)
terminal_value = (fcf_at_end * (1 + terminal_growth_rate)) / (discount_rate - terminal_growth_rate)

# Discount the terminal value back to the present value
terminal_value_discounted = terminal_value / ((1 + discount_rate) ** projection_years)

print("Terminal Value (undiscounted):", terminal_value)
print("Discounted Terminal Value:", terminal_value_discounted)

# %% [markdown]
"""
## 6. Compute Enterprise Value

The Enterprise Value (EV) is the sum of the discounted projected free cash flows and the discounted terminal value.
"""

# %%
# Sum of discounted free cash flows
sum_discounted_fcfs = sum([item["Discounted FCF"] for item in projected_fcfs])
enterprise_value = sum_discounted_fcfs + terminal_value_discounted

print("Sum of Discounted Free Cash Flows:", sum_discounted_fcfs)
print("Enterprise Value (EV):", enterprise_value)

# %% [markdown]
"""
## 7. Adjust for Net Debt to Find Equity Value

We retrieve total debt, total cash, and shares outstanding from the API (or use manual values) and calculate the net debt.
Then, subtract net debt from the enterprise value to get the equity value.
"""

# %%
# Retrieve debt, cash, and shares outstanding from the API
total_debt = info.get("totalDebt", None)
total_cash = info.get("totalCash", None)
shares_outstanding = info.get("sharesOutstanding", None)

# Use manual values if the API did not provide the data
if total_debt is None:
    if manual_total_debt is not None:
        total_debt = manual_total_debt
        print("Using manual total debt value:", total_debt)
    else:
        print("Warning: Total Debt data not available. Assuming 0.")
        total_debt = 0

if total_cash is None:
    if manual_total_cash is not None:
        total_cash = manual_total_cash
        print("Using manual total cash value:", total_cash)
    else:
        print("Warning: Total Cash data not available. Assuming 0.")
        total_cash = 0

if shares_outstanding is None:
    if manual_shares_outstanding is not None:
        shares_outstanding = manual_shares_outstanding
        print("Using manual shares outstanding value:", shares_outstanding)
    else:
        raise ValueError("Shares Outstanding data is not available via API. Please set 'manual_shares_outstanding' in the configuration.")

net_debt = total_debt - total_cash
print("Total Debt:", total_debt)
print("Total Cash:", total_cash)
print("Net Debt:", net_debt)
print("Shares Outstanding:", shares_outstanding)

# %% [markdown]
"""
## 8. Calculate Implied Stock Price (Fair Price)

The equity value is obtained by subtracting net debt from the enterprise value.
Dividing the equity value by the number of shares outstanding gives the implied stock price,
which represents the DCF-based fair price for the stock.
"""

# %%
equity_value = enterprise_value - net_debt
implied_stock_price = equity_value / shares_outstanding

print("Equity Value:", equity_value)
print("Implied Stock Price (DCF):", implied_stock_price)

# %% [markdown]
"""
## 9. DCF Analysis Summary

Below is a summary of the key metrics obtained from the DCF analysis.
"""

# %%
summary_data = {
    "Latest Free Cash Flow": latest_fcf,
    "Sum of Discounted Projected FCFs": sum_discounted_fcfs,
    "Discounted Terminal Value": terminal_value_discounted,
    "Enterprise Value": enterprise_value,
    "Net Debt": net_debt,
    "Equity Value": equity_value,
    "Shares Outstanding": shares_outstanding,
    "Implied Stock Price (Fair Price)": implied_stock_price
}

summary_df = pd.DataFrame(list(summary_data.items()), columns=["Metric", "Value"])
print(summary_df)

# %% [markdown]
"""
## 10. Fair Price Display

The following cell displays the DCF-based fair price for NVIDIA.
"""

# %%
print("\n======================================")
print("DCF-Based Fair Price for NVIDIA (NVDA): ${:.2f}".format(implied_stock_price))
print("======================================\n")


Company Name: NVIDIA Corporation
Sector: Technology
Industry: Semiconductors
Latest Free Cash Flow (most recent annual): 27021000000.0
   Year  Projected FCF  Discounted FCF
0     1   2.972310e+10    2.702100e+10
1     2   3.269541e+10    2.702100e+10
2     3   3.596495e+10    2.702100e+10
3     4   3.956145e+10    2.702100e+10
4     5   4.351759e+10    2.702100e+10
Terminal Value (undiscounted): 640330263304.2859
Discounted Terminal Value: 397594714285.71423
Sum of Discounted Free Cash Flows: 135105000000.0
Enterprise Value (EV): 532699714285.71423
Total Debt: 10014999552
Total Cash: 34800001024
Net Debt: -24785001472
Shares Outstanding: 24490000384
Equity Value: 557484715757.7142
Implied Stock Price (DCF): 22.763769171761
                             Metric         Value
0             Latest Free Cash Flow  2.702100e+10
1  Sum of Discounted Projected FCFs  1.351050e+11
2         Discounted Terminal Value  3.975947e+11
3                  Enterprise Value  5.326997e+11
4               